In [1]:
import matchms
import numpy as np
import json
from collections import namedtuple
from typing import List, Tuple


In [2]:
# load spectra, convert and write to json
matchms_spectra = list(matchms.importing.load_from_mgf("data/spectra.mgf"))
matchms.exporting.save_as_json(matchms_spectra, filename="data/spectra.json")

In [3]:
spectra = load_json_spectra("data/spectra.json")
spectra[0]

NameError: name 'load_json_spectra' is not defined

In [ ]:
print(get_min_max(spectra))
print(get_spectrum_ids(spectra)[0:10])

In [ ]:
# generate pairwise similarity matrix using ms2deepscore and modcos, save to numpy, load from file using numpy
from compMetabolomics.spectral_comparison import compute_similarities_cosine, compute_similarities_ms2ds, convert_similarity_to_distance

In [ ]:
if False:
  similarity_matrix_modcos = compute_similarities_cosine(matchms_spectra)
  similarity_matrix_ms2ds = compute_similarities_ms2ds(matchms_spectra, "models/ms2deepscore_model.pt")
  np.savetxt("data/similarities_modcos.npy", similarity_matrix_modcos)
  np.savetxt("data/similarities_ms2ds.npy", similarity_matrix_ms2ds)

In [ ]:
similarity_matrix_modcos = np.loadtxt("data/similarities_modcos.npy")
similarity_matrix_ms2ds = np.loadtxt("data/similarities_ms2ds.npy")

In [ ]:
# generate kmedoids clustering spectrum_id and group_id pairing
import compMetabolomics.kmedoid_clustering as kmc
kmedoid_grid = kmc.run_kmedoid_grid(convert_similarity_to_distance(similarity_matrix_ms2ds), [10, 20, 30, 50, 100, 150, 200, 250]) 
kmc.print_kmedoid_grid(kmedoid_grid)
kmc.plot_kmedoid_grid(kmedoid_grid)

In [ ]:
# tsne stuff

tsne_grid = tsne_embedding.run_tsne_grid(convert_similarity_to_distance(similarity_matrix_ms2ds), perplexity_values = [10,20,30,40,50])
tsne_embedding.plot_tsne_grid(tsne_grid)
tsne_embedding.print_tsne_grid(tsne_grid)
coordinates = tsne_embedding.extract_coordinates_from_entry(tsne_grid[3])
print(coordinates)

Please note that the quant table is already normalized by total ion current within sample. While this does not mitigate potential batch effects from different runs, it does put the data between samples onto the same scale.

In [ ]:
# list of feature_id, list of fold changes, list of node_size mappings, get fold change data per feature
import pandas as pd
quant_table = pd.read_csv("data/quant_table.csv")
treat_table = pd.read_csv("data/treat_table.csv")

In [ ]:
# This code block performs log2foldchange computations, alongside some formatting steps for styling and alignment
# Marge quantification and treatment tables
joined_data = pd.merge(quant_table, treat_table, on='sample_id', how="inner")
# drop sample_id column which is no longer needed in joined data
joined_data = joined_data.drop('sample_id', axis=1)
# drop treatment information from joined data to create feature quantificaiton exclusive data
numeric_feature_data = joined_data.drop('treatment', axis=1)
# Compute mean by feature and group combination (feature_id will be a pandas.df index)
mean_condition_0 = numeric_feature_data[joined_data['treatment'] == 'PleurotusOMSW0'].mean()
mean_condition_80 = numeric_feature_data[joined_data['treatment'] == 'PleurotusOMSW80'].mean()
# Merge pandas dataframes (using identical index)
meansdf = pd.concat([mean_condition_0, mean_condition_80], axis=1, join="inner")
# Rename columns
meansdf.columns = ["omsw0", "omsw80"]
# Compute ratio and log2 ratio
meansdf['ratio'] = meansdf['omsw80'] / meansdf['omsw0']
meansdf['log2ratio'] = np.log2(meansdf['ratio'])
# add feature id column and reset index
meansdf = meansdf.reset_index().rename(columns={'index': 'feature_id'})
# compute node size from log2fold change (linear mapping with bounds)
meansdf['node_size'] = [transform_log2_fold_change_to_node_size(val) for val in meansdf['log2ratio']]
# give columns more meaningful names
meansdf = meansdf.reset_index(drop=True).rename(columns={'omsw0': 'mean_omsw0', 'omsw80': 'mean_omsw80', 'ratio': 'ratio (omsw80 / omsw0)'})
# add increasing vs decreasing qualifier
meansdf["effect_direction"] = np.where((meansdf['mean_omsw80']-meansdf['mean_omsw0'])>=0, "positive", "negative")
# Align summary data data frame with spectrum id ordering usied elswhere
aligned_meansdf = align_with_spectral_feature_id(meansdf, spectra)
aligned_meansdf

In [ ]:
from typing import List
from compMetabolomics.spectrum import Spectrum
from compMetabolomics.construct_cytoscape_elements import generate_node_list

coordinates_table = tsne_embedding.extract_coordinates_from_entry(tsne_grid[3])
cluster_assignments = kmc.get_kmedoid_grid_entry_cluster_assignments(kmedoid_grid, 0)
node_list = generate_node_list(spectra=spectra, coordinates_table=coordinates_table, group_ids=cluster_assignments, summary_statistics_df=meansdf)
node_list

In [ ]:
import copy
tmp1 = copy.deepcopy(meansdf)
tmp2 = pd.DataFrame({'feature_id' : [spec.feature_id for spec in spectra]})
print(tmp2)

tmp2.merge(tmp1, how="inner")